<a href="https://colab.research.google.com/github/haedal-uni/analysis/blob/main/work/2025-04/Robo-Advisor/%ED%8F%89%EA%B7%A0%20%EB%B6%84%EC%82%B0%20%ED%8F%AC%ED%8A%B8%ED%8F%B4%EB%A6%AC%EC%98%A4%20%EC%B5%9C%EC%A0%81%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


해당 [링크](https://github.com/ald0met/RoboAdvisor_with_Python/blob/main/Chapter_3.ipynb)를 참고하여 코드 작성

평균-분산 최적화(Mean-Variance Optimization) 진행

PyPortfolioOpt 라이브러리를 사용하여 로보 어드바이저 기능 중 하나인 평균-분산 포트폴리오 최적화를 구현

단일 종목 데이터를 기반으로 수익률 계산 후 평균-분산 최적화 기법을 통해 비중(weight) 을 도출

In [4]:
pip install PyPortfolioOpt

In [5]:
import pandas as pd
from typing import Optional, Dict
from pypfopt import EfficientFrontier

### 데이터 불러오기 함수
def load_and_clean_data(file_path: str, ticker: str) -> pd.DataFrame:
    df = pd.read_csv(file_path, parse_dates=['날짜'], index_col="날짜", thousands=",")
    df['거래량'] = df['거래량'].apply(lambda x: float(x.replace('K', '')) * 1000 if isinstance(x, str) and 'K' in x else float(x))
    df['변동 %'] = df['변동 %'].apply(lambda x: float(x.replace('%', '')) / 100 if isinstance(x, str) else x)
    df = df.sort_index().dropna()
    df = df[['종가']]  # 종가만 남기기
    df.columns = [ticker]  # 종가 → 티커명
    return df

### 수익률 계산 함수
def calculate_return(price_data: pd.DataFrame) -> pd.DataFrame:
    return price_data.pct_change(1) * 100  # 일간 수익률

### 평균-분산 최적화 함수
def get_mean_variance_weights(return_data: pd.DataFrame, risk_aversion: int) -> Optional[Dict]:
    expected_return = return_data.mean(skipna=False).to_list()
    cov = return_data.cov(min_periods=len(return_data))

    if cov.isnull().values.any() or cov.empty:
        return None

    ef = EfficientFrontier(
        expected_returns=expected_return,
        cov_matrix=cov,
        solver="OSQP"
    )
    ef.max_quadratic_utility(risk_aversion=risk_aversion)
    weights = dict(ef.clean_weights(rounding=None))
    return weights

### 데이터 결합 및 실행
# 각각 데이터 로딩
steel = load_and_clean_data('미국 철강 코일 선물 과거 데이터.csv', '철강')
copper = load_and_clean_data('구리 선물 과거 데이터.csv', '구리')

# 날짜 기준 병합 (inner join으로 공통 날짜만)
merged_prices = pd.concat([steel, copper], axis=1, join="inner")

# 수익률 계산
returns = calculate_return(merged_prices).dropna()

# 평균-분산 포트폴리오 최적화
weights = get_mean_variance_weights(returns, risk_aversion=1)

print("최적 포트폴리오 비중:")
print(weights)


최적 포트폴리오 비중:
{'철강': 0.3067782708400018, '구리': 0.6932217291599982}


|자산|투자 비중 (%)|
|------|---|
|철강|약 30.68%|
|구리|약 69.32%|

철강과 구리 두 자산에 투자할 때 수익률을 최대화하면서 위험(변동성)을 최소화하는 최적의 자산 배분

구리에 더 많이 투자하고 철강은 조금 투자하라는 뜻

<br><br>

총 자금이 1,000만 원이라면?

철강에 약 3,068,000원

구리에 약 6,932,000원 투자
